# Load libraries

In [1]:
import pickle, logging
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,SimpleRNN, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load data

In [2]:
reviews=pd.read_csv('../data/amazon_reviews_us_Apparel_v1_00.tsv.gz.tsv',sep='\t')

In [3]:
reviews['label']=(reviews['verified_purchase']=='y').astype(int)

In [4]:
reviews=reviews.dropna()

In [5]:
reviews['label'].mean()

0.8995045187685107

# Samples

In [6]:
def get_samples(df, n):
    samples=df[:n]
    print('Initial label mean: {0}'.format(samples['label'].mean()))
    counts=samples.groupby('label')['label'].count().to_dict()
    sorted_counts = sorted(counts.items(), key=lambda kv: kv[1])
    undersample = samples[samples['label'] == sorted_counts[1][0]].sample(sorted_counts[0][1])
    samples=pd.concat([undersample, samples[samples['label'] == sorted_counts[0][0]]])
    print('Undersampled label mean: {0}'.format(samples['label'].mean()))
    return samples

In [7]:
def clean_text(df,features=['review_body','label']):
    df['review_body'].replace(regex=[r'([^a-z\s]+)'], value='', inplace=True)
    return df[features].copy()

In [8]:
samples=get_samples(reviews, n=5000)
samples=clean_text(samples)

Initial label mean: 0.6492
Undersampled label mean: 0.5


In [10]:
samples['review_body'].head()

4745    it sounded like a good idea  yes its flatter t...
541                            nice outfit just too small
1417    i thought they were like the flyless tagless j...
3459    i ordered this suit up  sizes and it fits well...
3458                                   very cutei love it
Name: review_body, dtype: object

## Save/Load the clean dataframe

In [11]:
# saved_dataframe='df_verified_purchase.p'
# pickle.dump( samples, open( saved_dataframe, "wb" ) )
# samples= pickle.load( open( saved_dataframe, "rb" ) )

# Split data

In [12]:
def split_pad(selected_reviews):
    X_train, X_test, Y_train, Y_test = train_test_split(selected_reviews['review_body'].values,\
                                                    selected_reviews['label'].values,\
                                                    test_size=0.10,\
                                                    shuffle=True)
    for label in [Y_train, Y_test]:
          logging.debug("label mean: {0}".format(label.mean()))
    tokenizer = Tokenizer(num_words= 50000)
    tokenizer.fit_on_texts(X_train)
    X_train_tok = tokenizer.texts_to_sequences(X_train)
    X_test_tok = tokenizer.texts_to_sequences(X_test)
    max_review_length = 600
    X_train_pad = sequence.pad_sequences(X_train_tok, maxlen=max_review_length)
    X_test_pad = sequence.pad_sequences(X_test_tok, maxlen=max_review_length)
    return X_train_pad, X_test_pad, Y_train, Y_test

# Compile models

In [13]:
models=[]

max_review_length = 600
embedding_vector_length = 32
top_words = 50000
models.append(Sequential())
model=models[-1]
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(SimpleRNN(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [14]:
embedding_vector_length = 40
models.append(Sequential())
model=models[-1]
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [15]:
models.append(Sequential())
model=models[-1]
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(GRU(256, return_sequences=True))
model.add(SimpleRNN(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [17]:
samples=get_samples(reviews, n=5000)
samples=clean_text(samples)
X_train_pad, X_test_pad, Y_train, Y_test=split_pad(samples)
for model in models:
    model.fit(X_train_pad, Y_train, epochs=100, callbacks=[callback], batch_size=64,validation_data=(X_test_pad,Y_test))
    # Final evaluation of the model on test data
    scores = model.evaluate(X_test_pad, Y_test, verbose=0)
    print("loss: {0:2.2f}, accuracy {1:2.2f}".format(scores[0],scores[1]))

Train on 3157 samples, validate on 351 samples
Epoch 1/100
3157/3157 [==============================] - 14s 5ms/sample - loss: 0.6596 - accuracy: 0.6104 - recall: 0.6181 - precision: 0.6089 - val_loss: 0.6066 - val_accuracy: 0.6724 - val_recall: 0.6000 - val_precision: 0.7000
Epoch 2/100
3157/3157 [==============================] - 13s 4ms/sample - loss: 0.6095 - accuracy: 0.6554 - recall: 0.6941 - precision: 0.6443 - val_loss: 0.6449 - val_accuracy: 0.5242 - val_recall: 0.0629 - val_precision: 0.7857
Epoch 3/100
3157/3157 [==============================] - 12s 4ms/sample - loss: 0.5937 - accuracy: 0.6864 - recall: 0.7612 - precision: 0.6623 - val_loss: 0.6818 - val_accuracy: 0.6211 - val_recall: 1.0000 - val_precision: 0.5682
Epoch 4/100
3157/3157 [==============================] - 13s 4ms/sample - loss: 0.6082 - accuracy: 0.6741 - recall: 0.7859 - precision: 0.6423 - val_loss: 0.5991 - val_accuracy: 0.6524 - val_recall: 0.8743 - val_precision: 0.6047
Epoch 5/100
3157/3157 [==========